# Instructor Do: Intro to VADER Sentiment

In [1]:
# Initial imports
import os
from pathlib import Path
import pandas as pd
from newsapi import NewsApiClient
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from dotenv import load_dotenv
load_dotenv("newsapi.env")


True

In [2]:
# Download/Update the VADER Lexicon
nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()



[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\josh\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
# Reading the News API key enviroment variable
api_key = os.getenv("news_api")



In [4]:
type(api_key)

str

In [58]:
%pwd

'C:\\Users\\josh\\OneDrive\\Desktop\\NW_Fintech_Project3'

In [5]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)



In [6]:
news_df = pd.DataFrame(index = pd.RangeIndex(start = 0, stop = 100))
#payload = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
#first_table = payload[0]
#second_table = payload[1]

#sp500 = first_table
sp500 = ['AAPL', 'MSFT']

#companies = sp500['Security'].tolist()
#companies = sp500['AAPL', 'MSFT'].tolist()

In [7]:
sp500

['AAPL', 'MSFT']

In [8]:
# Fetch all the news 
    
for company in sp500:
    news = newsapi.get_everything(from_param='2022-03-09',
to='2022-04-08',
        q=company,
        language="en",
        page_size=100,
    )
    contents = []
    print('done')
    for article in news['articles']:
        contents.append(article)
    
    news_df[company] = pd.Series(contents)
# Print total articles

news_df
news_df.dtypes

done
done


AAPL    object
MSFT    object
dtype: object

In [9]:
news

{'status': 'ok',
 'totalResults': 546,
 'articles': [{'source': {'id': None, 'name': 'StockNews.com'},
   'author': 'Jaimini Desai',
   'title': '3 Dividend Growth Stocks to Buy During This Market Correction',
   'description': 'One characteristic of a high-quality stock is a consistent track record of hiking dividends. This indicates a strong business that can thrive in all types of economic conditions, and a...',
   'url': 'https://stocknews.com/news/msft-cost-abbv-3-dividend-growth-stocks-to-buy-during-this-market-correction/',
   'urlToImage': 'https://assets.entrepreneur.com/providers/stocknews/hero-image-stocknews-422729.jpeg',
   'publishedAt': '2022-03-21T13:14:05Z',
   'content': 'One characteristic of a high-quality stock is a consistent track record of hiking dividends. This indicates a strong business that can thrive in all types of economic conditions, and a management tea… [+7647 chars]'},
  {'source': {'id': None, 'name': 'StockNews.com'},
   'author': 'Tim Biggam',
   '

In [14]:
news_df['AAPL'][0]

{'source': {'id': None, 'name': 'StockNews.com'},
 'author': 'Subhasree Kar',
 'title': 'Is Apple Stock a Buy at $175?',
 'description': 'Shares of technology giant Apple (AAPL) have been gaining in price lately, fueled by news that indicates impressive demand for its iPhone 13 globally. However, the EU is outlining new...',
 'url': 'https://stocknews.com/news/aapl-alot-lnvgy-ttdky-is-apple-stock-a-buy-at-175/',
 'urlToImage': 'https://assets.entrepreneur.com/providers/stocknews/hero-image-stocknews-423244.jpeg',
 'publishedAt': '2022-03-28T12:37:54Z',
 'content': 'Shares of technology giant Apple (AAPL) have been gaining in price lately, fueled by news that indicates impressive demand for its iPhone 13 globally. However, the EU is outlining new tech regulation… [+6318 chars]'}

In [54]:
# Create the Facebook Libra sentiment scores DataFrame
libra_sentiments = []

for col in news_df.columns:
    column = news_df[col]
    symbol_sentiments = []
    for article in column:
        
        try:
            text = article["content"]
            date = article["publishedAt"][:10]
            sentiment = analyzer.polarity_scores(text)
            compound = sentiment["compound"]
            pos = sentiment["pos"]
            neu = sentiment["neu"]
            neg = sentiment["neg"]


            symbol_sentiments.append({
                "text": text,
                "symbol" : col,
                "date": date,
                "compound": compound,
                "positive": pos,
                "negative": neg,
                "neutral": neu})
            

        except AttributeError:
            pass
    libra_sentiments.append(symbol_sentiments)
for i in range(len(libra_sentiments)):
    libra_sentiments[i] = pd.DataFrame(libra_sentiments[i])
    
# Create DataFrame
libra_df = pd.concat(objs=libra_sentiments)

# Reorder DataFrame columns
cols = ["date", 'symbol',"text", "compound", "positive", "negative", "neutral"]
libra_df = libra_df[cols]

libra_df.head()



,date,symbol,text,compound,positive,negative,neutral
0,2022-03-28,AAPL,Shares of technology giant Apple (AAPL) have b...,0.7783,0.209,0.038,0.754
1,2022-03-20,AAPL,Two tried and true ways to tame market uncerta...,0.7269,0.203,0.057,0.740
2,2022-04-04,AAPL,Apple’s next move may be an iPhone hardware su...,0.0772,0.125,0.062,0.813
3,2022-03-17,AAPL,Despite the expectation that the Fed will rais...,0.5267,0.093,0.000,0.907
4,2022-03-22,AAPL,Follow The Analysts To Three Winning Stocks \r...,0.7351,0.165,0.035,0.800


In [ ]:
# Get descriptive stats from the DataFrame
libra_df.describe()

